In [1]:
import random
import operator
import numpy

from deap import algorithms
from deap import base
from deap import tools
from deap import creator
from deap import gp

In [2]:
def if_then_else(condition, out1, out2):
    return out1 if condition else out2

In [7]:
## Initialize Multiplexer problem input and output vectors

MUX_SELECT_LINES = 3
MUX_IN_LINES = 2 ** MUX_SELECT_LINES
MUX_TOTAL_LINES = MUX_SELECT_LINES + MUX_IN_LINES

# input: [A0 A1 A2 D0 D1 D2 D3 D4 D5 D6 D7] for a 8-3 mux
inputs = [[0] * MUX_TOTAL_LINES for i in range(2 ** MUX_TOTAL_LINES)]
outputs = [None] * (2 ** MUX_TOTAL_LINES)

for i in range(2 ** MUX_TOTAL_LINES):
    value = i
    divisor = 2 ** MUX_TOTAL_LINES
    # Fill the inputs bits
    for j in range(MUX_TOTAL_LINES):
        divisor /= 2
        if value >= divisor:
            inputs[i][j] = 1
            value -= divisor
    # Determine the corresponding output
    indexOutput = MUX_SELECT_LINES
    for j, k in enumerate(inputs[i][:MUX_SELECT_LINES]):
        indexOutput += k * 2**j
    outputs[i] = inputs[i][indexOutput]

In [8]:
pset = gp.PrimitiveSet("MAIN", MUX_TOTAL_LINES, prefix="IN")
pset.addPrimitive(operator.and_, 2)
pset.addPrimitive(operator.or_, 2)
pset.addPrimitive(operator.not_, 1)
pset.addPrimitive(operator.not_, 1)
pset.addPrimitive(if_then_else, 3)
pset.addTerminal(1)
pset.addTerminal(0)

In [9]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

In [11]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=2, max_=4)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [12]:
def evaluate(individual):
    func = toolbox.compile(expr=individual)
    return sum(func(*in_) == out for in_, out in zip(inputs, outputs)),

In [13]:
toolbox.register("evaluate",evaluate)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genGrow, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [15]:
def main():
    pop = toolbox.population(n=40)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, cxpb=0.8, mutpb=0.1, ngen=40, stats=stats, halloffame=hof, verbose=True)
    
    return pop, stats, hof


In [16]:
pop, stats, hof = main()

gen	nevals	avg   	std    	min	max 
0  	40    	1038.6	70.0146	896	1184
1  	34    	1086.6	85.3583	896	1184
2  	28    	1151  	56.0054	896	1216
3  	33    	1161.6	35.9199	1056	1216
4  	34    	1176  	33.6095	1024	1216
5  	30    	1171  	52.9358	928 	1216
6  	36    	1195.6	31.9975	1088	1280
7  	29    	1204  	50.4381	1024	1280
8  	34    	1217.6	55.4025	1088	1280
9  	37    	1235.2	51.9938	1152	1280
10 	26    	1238.4	65.2674	1024	1280
11 	38    	1229.6	58.0813	1120	1280
12 	32    	1228  	70.722 	1024	1312
13 	37    	1229.6	57.193 	1024	1312
14 	37    	1232.4	54.0799	1152	1296
15 	37    	1239.2	54.1346	1152	1296
16 	34    	1254.4	66.4337	1056	1312
17 	35    	1266.8	49.6564	1096	1312
18 	29    	1288.8	36.4768	1168	1344
19 	26    	1289  	44.0068	1152	1344
20 	30    	1297.2	54.6567	1152	1344
21 	36    	1299.2	60.3138	1152	1344
22 	34    	1303  	61.0623	1024	1344
23 	37    	1312.8	49.1789	1152	1360
24 	32    	1330.6	42.0433	1152	1376
25 	32    	1341.6	40.7215	1216	1424
26 	27    	1337.4	63.2174	1088	1